<a href="https://colab.research.google.com/github/ashishpatel26/tcn-keras-Examples/blob/master/TCN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tcn

In [2]:
import numpy as np
import tensorflow as tf
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tcn import compiled_tcn

In [3]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.97):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True


callbacks = myCallback()

In [4]:
def data_generator():
    # input image dimensions
    img_rows, img_cols = 28, 28
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(-1, img_rows * img_cols, 1)
    x_test = x_test.reshape(-1, img_rows * img_cols, 1)

    num_classes = 10
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    y_train = np.expand_dims(y_train, axis=2)
    y_test = np.expand_dims(y_test, axis=2)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    return (x_train, y_train), (x_test, y_test)

In [5]:
(x_train, y_train), (x_test, y_test) = data_generator()

In [6]:
 model = compiled_tcn(return_sequences=False,
                         num_feat=1,
                         num_classes=10,
                         nb_filters=20,
                         kernel_size=6,
                         dilations=[2 ** i for i in range(9)],
                         nb_stacks=1,
                         max_len=x_train[0:1].shape[1],
                         use_skip_connections=True)

Metal device set to: Apple M1 Pro


2022-10-27 15:08:38.870684: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-27 15:08:38.870934: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


x.shape= (None, 20)
model.x = (None, 784, 1)
model.y = (None, 10)


/Users/sjkang/miniforge3/envs/tensorflow/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
print(f'x_train.shape = {x_train.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'x_test.shape = {x_test.shape}')
print(f'y_test.shape = {y_test.shape}')

x_train.shape = (60000, 784, 1)
y_train.shape = (60000, 10, 1)
x_test.shape = (10000, 784, 1)
y_test.shape = (10000, 10, 1)


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784, 1)]          0         
                                                                 
 tcn (TCN)                   (None, 20)                41320     
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 activation (Activation)     (None, 10)                0         
                                                                 
Total params: 41,530
Trainable params: 41,530
Non-trainable params: 0
_________________________________________________________________


In [9]:

model.fit(x_train, y_train.squeeze().argmax(axis=1), epochs=100,
          validation_data=(x_test, y_test.squeeze().argmax(axis=1)),callbacks=[myCallback()])

Epoch 1/100


2022-10-27 15:08:57.462010: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-27 15:08:58.789051: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - ETA: 0s - loss: 0.3396 - accuracy: 0.8957

2022-10-27 15:13:49.335779: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 311s 165ms/step - loss: 0.3396 - accuracy: 0.8957 - val_loss: 0.1078 - val_accuracy: 0.9656
Epoch 2/100
1875/1875 [==============================] - 300s 160ms/step - loss: 0.1356 - accuracy: 0.9591 - val_loss: 0.0944 - val_accuracy: 0.9711
Epoch 3/100
1875/1875 [==============================] - 315s 168ms/step - loss: 0.1097 - accuracy: 0.9663 - val_loss: 0.0853 - val_accuracy: 0.9732
Epoch 4/100
1875/1875 [==============================] - ETA: 0s - loss: 0.0973 - accuracy: 0.9706
Reached 99% accuracy so cancelling training!
1875/1875 [==============================] - 308s 164ms/step - loss: 0.0973 - accuracy: 0.9706 - val_loss: 0.0827 - val_accuracy: 0.9740
